In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load or Create Simulated CDM Data (for demo - use real CSV in production)
data = [
    {'event_id': 1, 'creation_date': '2025-11-01', 'miss_distance': 1500, 'collision_probability': 1e-5, 'object1_name': 'STARLINK-1001', 'object2_name': 'STARLINK-2002', 'tca': '2025-11-02T10:15', 'relative_position_r': 500, 'relative_position_t': 100, 'relative_position_n': 50},
    {'event_id': 1, 'creation_date': '2025-11-02', 'miss_distance': 1000, 'collision_probability': 2e-5, 'object1_name': 'STARLINK-1001', 'object2_name': 'STARLINK-2002', 'tca': '2025-11-02T10:15', 'relative_position_r': 300, 'relative_position_t': 120, 'relative_position_n': 60},
    {'event_id': 2, 'creation_date': '2025-11-01', 'miss_distance': 2500, 'collision_probability': 1e-6, 'object1_name': 'ONEWEB-100', 'object2_name': 'STARLINK-2002', 'tca': '2025-11-03T09:10', 'relative_position_r': 1200, 'relative_position_t': -50, 'relative_position_n': -10},
    # Historical events
    {'event_id': 1, 'creation_date': '2022-11-01', 'miss_distance': 2500, 'collision_probability': 5e-6, 'object1_name': 'STARLINK-1001', 'object2_name': 'STARLINK-2002', 'tca': '2022-11-02T10:15', 'relative_position_r': 900, 'relative_position_t': 250, 'relative_position_n': 120},
]

df = pd.DataFrame(data)

# 2. Extract Latest CDM per Event
latest_cdm = df.sort_values('creation_date').groupby('event_id').tail(1)

# 3. Compare Latest to Previous CDMs for Same Event
def compare_event(event_id):
    event_cdms = df[df['event_id'] == event_id].sort_values('creation_date')
    return event_cdms

# 4. Compare with Historical Events (by miss_distance threshold)
def find_similar_historical(event_id, threshold=500):
    md = latest_cdm.loc[latest_cdm['event_id'] == event_id, 'miss_distance'].values[0]
    similar = df[(df['miss_distance'] > md - threshold) & (df['miss_distance'] < md + threshold) & (df['event_id'] != event_id)]
    return similar

# 5. Visualize Miss Distance Over Time for an Event
def plot_event_trend(event_id):
    event_cdms = compare_event(event_id)
    plt.figure(figsize=(8,4))
    plt.plot(event_cdms['creation_date'], event_cdms['miss_distance'], marker='o')
    plt.title(f"Miss Distance Trend for Event {event_id}")
    plt.xlabel('Date')
    plt.ylabel('Miss Distance (m)')
    plt.grid(True)
    plt.show()

# 6. Position Visualization (r, t, n vectors at TCA)
def plot_positions(event_id):
    event_cdms = compare_event(event_id)
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(event_cdms['relative_position_r'], event_cdms['relative_position_t'], event_cdms['relative_position_n'], c='b', marker='o')
    ax.set_xlabel('Radial (R)')
    ax.set_ylabel('Transverse (T)')
    ax.set_zlabel('Normal (N)')
    plt.title(f'Position Vectors at TCA for Event {event_id}')
    plt.show()

# 7. Example Usage
for eid in latest_cdm['event_id']:
    print(f"\n--- Event {eid} Latest CDM ---")
    print(latest_cdm[latest_cdm['event_id'] == eid])
    print("\nComparing with previous CDMs:")
    print(compare_event(eid))
    print("\nSimilar Historical Events:")
    print(find_similar_historical(eid))
    plot_event_trend(eid)
    plot_positions(eid)

# 8. Operator Dashboard Summary (tabular)
dashboard = latest_cdm[['event_id', 'creation_date', 'object1_name', 'object2_name', 'miss_distance', 'collision_probability', 'tca']]
print("\n=== Latest CDM Dashboard Summary ===")
print(dashboard)

# 9. To extend: integrate CSV or XML CDM parsing, interactive Dash/Panel web UI, real operator metadata
